## This was very confusing for me, but the steps below seem to be helpful

https://groups.google.com/forum/#!msg/strava-api/XbQJrQOm9Dg/rNnc2cL3DAAJ

1. first i need to authenticate - authentication will happen by me going to this website:
https://www.strava.com/oauth/authorize?redirect_uri=http://localhost:5744/&response_type=code&scope=read,read_all,activity:read,activity:read_all,profile:read_all&client_id=15756
2. It will send me to a local host, so then i need to get the "code" value and store it
3. Then i need to post it to the authentication site, using the code, my client id, and client secret so i can pull it
POST https://www.strava.com/oauth/token?grant_type=authorization_code&client_id=<YOUR CLIENT ID>&client_secret=<YOUR CLIENT SECRET>&code=<code>



In [2]:
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as BS
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException,NoSuchElementException
import configparser

In [13]:
config = configparser.ConfigParser()
config.read("../strava.ini")
params={'client_id':config['strava']['client_id'],
        'client_secret':config['strava']['client_secret'],
        'code':config['strava']['code']}
auth_url=config['strava']['auth_url']
ref_url=config['strava']['ref_url']
athlete_id=config['strava']['athlete_id']

In [14]:
config['strava']['client_id']

'15756'

In [15]:
r_auth = requests.post(auth_url, data=params)
print(r_auth.status_code)
response=r_auth.json()

400


In [12]:
response

{'access_token': '40ddb954f220854ca19ebe1ebb788bb5dd7f3e2e',
 'athlete': {'badge_type_id': 0,
  'city': 'Washington',
  'country': 'United States',
  'created_at': '2016-07-01T12:05:32Z',
  'firstname': 'Ira',
  'follower': None,
  'friend': None,
  'id': 16081751,
  'lastname': 'Rickman',
  'premium': False,
  'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/16081751/4498868/2/large.jpg',
  'profile_medium': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/16081751/4498868/2/medium.jpg',
  'resource_state': 2,
  'sex': 'M',
  'state': 'District of Columbia',
  'summit': False,
  'updated_at': '2020-05-09T18:06:07Z',
  'username': None},
 'expires_at': 1590139266,
 'expires_in': 21600,
 'refresh_token': '3af76bbca07e8d016006062460329834731868c2',
 'token_type': 'Bearer'}

In [6]:
r_ref = requests.post(ref_url, data=params)
print(r_ref.status_code)
ref_response=r_ref.json()

400


In [56]:
import time
import datetime
import swagger_client
from swagger_client.rest import ApiException
from pprint import pprint

In [11]:
### need to figure out how to pull all my activities over time

In [12]:
configuration=swagger_client.Configuration()
configuration.access_token=response['access_token']
api_instance = swagger_client.AthletesApi(swagger_client.ApiClient(configuration))

try:
    # Get stats
    api_response = api_instance.get_stats(id=athlete_id)
    pprint(api_response)
except ApiException as e:
    print("Exception when calling AthletesApi->get_logged_in_athlete: %s\n" % e)

{'all_ride_totals': {'achievement_count': None,
                     'count': 45,
                     'distance': 1684605.0,
                     'elapsed_time': 276979,
                     'elevation_gain': 6155.0,
                     'moving_time': 232803},
 'all_run_totals': {'achievement_count': None,
                    'count': 475,
                    'distance': 4208496.0,
                    'elapsed_time': 1500814,
                    'elevation_gain': 27155.0,
                    'moving_time': 1329110},
 'all_swim_totals': {'achievement_count': None,
                     'count': 45,
                     'distance': 58605.0,
                     'elapsed_time': 101728,
                     'elevation_gain': 0.0,
                     'moving_time': 80302},
 'biggest_climb_elevation_gain': 139.7,
 'biggest_ride_distance': 110666.0,
 'recent_ride_totals': {'achievement_count': 0,
                        'count': 0,
                        'distance': 0.0,
                  

In [25]:
## getting latest date
strav=pd.read_csv("../data/my_actvities.csv")

In [51]:
last_date=datetime.datetime.strptime(strav.start_date.max().split(' ')[0],"%Y-%m-%d")
new_timestamp=last_date.timestamp
delta=datetime.datetime.now() - last_date
delta.days

6

In [30]:
timestamp = (datetime.datetime(2018, 5, 15)).timestamp()
timestamp 

1526356800.0

In [16]:
configuration = swagger_client.Configuration()
configuration.access_token = response['access_token']

# create an instance of the API class
api_instance = swagger_client.ActivitiesApi(swagger_client.ApiClient(configuration))

try:
    # Get Authenticated Athlete
    api_response = api_instance.get_logged_in_athlete_activities(after=timestamp, per_page=30,page=2)
    pprint(api_response)
except ApiException as e:
    print("Exception when calling AthletesApi->get_logged_in_athlete: %s\n" % e)

NameError: name 'swagger_client' is not defined

In [63]:
pd.options.display.max_columns=100

In [62]:
len(api_response)

30

In [58]:
len(api_response)
type(api_response[len(api_response)-1])
example=list(api_response[150].to_dict().keys())
example=[x for x in example if x not in ['map','athlete','start_latlng','end_latlng']]
dicts={}
for n in range(len(api_response)):
    d=api_response[n].to_dict()
    new_dict={variable:d[variable] for variable in example}
    dicts[n]=new_dict

In [60]:
type(api_response)

list

In [69]:
index=list(dicts.keys())
all_activities=pd.DataFrame([dicts[key] for key in index],index=index)
all_activities.to_csv('../Data/my_actvities.csv',index=False)

In [52]:
api_response

NameError: name 'api_response' is not defined

In [18]:
import requests
import pandas as pd

In [19]:
all_activities.type.value_counts()

Run       133
Ride       38
Swim       27
Rowing      1
Walk        1
Name: type, dtype: int64

##### Steps
1. Pull Whoop Data - labels should be red, yellow, green
2. Look at how much data i have (split off into activity types)
3. Brainstorm features - spend a lot of time on this
    a. Time distribution of strain and recovery
    b. time since last workout
    c. speed * distance
    d. be creative!
4. Create observation level data

##### Backlog
1. Build model
2. Automatically pull most recent data using AWS

In [187]:
w='https://app.whoop.com/login'

In [200]:
browser=webdriver.Chrome('/usr/local/bin/chromedriver')
browser.get(w)

In [201]:
username=browser.find_element_by_name('username')
password=browser.find_element_by_name('password')
username.send_keys(config['whoop']['username'])
password.send_keys(config['whoop']['pass'])
submit=browser.find_element_by_name('form')
submit.click()

In [93]:
strain_click=browser.find_elements_by_class_name('score')[0]
strain_click.click()

In [23]:
strain_amt=browser.find_element_by_class_name("workout-strain")
strain_amt.get_attribute('innerHTML')
strain_amt.find_element_by_tag_name("strong").text

'16.2'

In [24]:
buttons=browser.find_elements_by_tag_name('button')
#[x for x in [span.find_elements_by_tag_name('span')  
[x.text for item in [x for x in [span.find_elements_by_tag_name('span') for span in [b for b in buttons if b.get_attribute("ng-click")=="click($event, activity.id)"]]] for x in item]


['Running', '16.2']

In [25]:


for x in buttons:
    if x.get_attribute("ng-click")=="click($event, activity.id)":
        spans=x.find_elements_by_tag_name("span")
        print([x.text for x in spans], x)
    else:
        print("nothing", x)

nothing <selenium.webdriver.remote.webelement.WebElement (session="08fde42d8d6815291fceb7d2092c9f08", element="0.7561134548339705-10")>
nothing <selenium.webdriver.remote.webelement.WebElement (session="08fde42d8d6815291fceb7d2092c9f08", element="0.7561134548339705-11")>
['Running', '16.2'] <selenium.webdriver.remote.webelement.WebElement (session="08fde42d8d6815291fceb7d2092c9f08", element="0.7561134548339705-12")>
nothing <selenium.webdriver.remote.webelement.WebElement (session="08fde42d8d6815291fceb7d2092c9f08", element="0.7561134548339705-13")>
nothing <selenium.webdriver.remote.webelement.WebElement (session="08fde42d8d6815291fceb7d2092c9f08", element="0.7561134548339705-14")>
nothing <selenium.webdriver.remote.webelement.WebElement (session="08fde42d8d6815291fceb7d2092c9f08", element="0.7561134548339705-15")>
nothing <selenium.webdriver.remote.webelement.WebElement (session="08fde42d8d6815291fceb7d2092c9f08", element="0.7561134548339705-16")>
nothing <selenium.webdriver.remote.w

In [26]:
new_strain=["null"]*6
new_strain

['null', 'null', 'null', 'null', 'null', 'null']

In [142]:
def go_to_strain():
    work=0
    try:
        strain_click=browser.find_elements_by_class_name('score')[0]
        strain_click.click()
        work=1
    except StaleElementReferenceException:
        work=0

In [143]:
## goal function is tos look for the activity - if it doesn't exist, then look for "no activity" 
## store the result, if the old activity strain is the same as the new

def get_activity_strain(old_strain,old_date):
    work=0
    while work==0:
        try:
            buttons=browser.find_elements_by_tag_name('button')
            new_strain=[x.text for item in [x for x in [span.find_elements_by_tag_name('span') for span in [b for b in buttons if b.get_attribute("ng-click")=="click($event, activity.id)"]]] for x in item]
            if len(new_strain)==0:
                new_strains=["null"]*6
            elif len(new_strain)<7:
                new_strains=new_strain+['null']*(6-len(new_strain))
            else:
                new_strains=new_strain
            if old_strain!=new_strains:
                return new_strains
                work=1
            elif old_date!=get_date(old_date):
                return new_strains
                work=1
        except StaleElementReferencException:
            work=0

In [184]:
def get_date(old_date):
    work=0
    while work==0:
        try:
            #new_day=browser.find_element_by_class_name('datepicker--label')
            new_day=WebDriverWait(browser, 3).until(EC.presence_of_element_located((By.CLASS_NAME, 'datepicker--label')))
            new_date=new_day.text
            if new_date!=old_date:
                return [new_date]
                work=1
        except StaleElementReferenceException:
            work=0

In [181]:
def get_scores(old_scores):
    work=0
#     new_scores=browser.find_elements_by_class_name('score')
#     new_score_list=[x.text for x in new_scores]
#     if new_score_list!=old_scores:
#         return new_score_list
#     else:
#         return ['it','didn','t','work','bud']
    while work==0:
        try:
            new_scores=browser.find_elements_by_class_name('score')
            new_score_list=[x.text for x in new_scores]
            if new_score_list!=old_scores:
                return new_score_list
                work=1
            else:
                work=0
        except StaleElementReferenceException:
            work=0
            

In [146]:
def get_button():
    work=0
    while work==0:
        try: 
            back=browser.find_element_by_class_name('datepicker--prev')
            back.click()
            work=1
        except StaleElementReferenceException:
            work=0

In [82]:
browser.current_url

'https://app.whoop.com/athlete/24590/strain/1d/2019-03-29/16266225'

In [73]:
browser.get('https://app.whoop.com/athlete/24590/strain/1d/2019-03-29')

In [80]:
tod=datetime.datetime.today()-datetime.timedelta(1)
tod.strftime('%Y-%m-%d')

'2019-03-31'

In [38]:
num=1
(get_date('today')[0].strip().lower()!='wed, may 30th' and num<300)

False

In [167]:
whoop=pd.DataFrame(columns=['strain','recovery','sleep_perf','sleep','rec_sleep','date', 
                            'activity_1','activity_1_score','activity_2','activity_2_score',
                           'activity_3','activity_3_score'])
num=0
old_date='today'
old_scores=['none']
old_strain=['null']
go_to_strain()
new_date=get_date(old_date)
while (new_date[0].strip().lower()!='wed, mar 27th' and num<20):
    new_date=get_date(old_date)
    new_score=get_scores(old_scores)
    strains=get_activity_strain(old_strain,old_date)
    old_date=new_date
    old_scores=new_score
    old_strain=strains
    whoop.loc[len(whoop)]=new_score[:5]+new_date+strains
    get_button()
    num+=1

here
here
here
here
here
here
here
here
here
here
here
here


In [47]:
whoop

,strain,recovery,sleep_perf,sleep,rec_sleep,date,activity_1,activity_1_score,activity_2,activity_2_score,activity_3,activity_3_score
0,18.2,63%,100%,9:23,9:11,"Today, Mar 24th",Running,15.6,Running,9.8,null,null
1,8.6,27%,66%,6:39,10:07,"Sat, Mar 23rd",null,null,null,null,null,null
2,19.5,62%,75%,7:23,9:47,"Fri, Mar 22nd",Running,18.6,null,null,null,null
3,18.0,30%,78%,7:16,9:18,"Thu, Mar 21st",Running,17.2,null,null,null,null
4,12.0,25%,75%,6:56,9:16,"Wed, Mar 20th",null,null,null,null,null,null
5,15.8,21%,83%,6:56,8:23,"Tue, Mar 19th",Running,14.2,null,null,null,null
6,15.4,24%,100%,9:16,8:12,"Mon, Mar 18th",Hiking/Rucking,12.2,null,null,null,null
7,14.5,17%,46%,3:28,7:35,"Sun, Mar 17th",Running,12.1,null,null,null,null
8,15.3,13%,71%,7:10,10:08,"Sat, Mar 16th",Running,12.4,null,null,null,null
9,18.3,72%,64%,6:00,9:21,"Fri, Mar 15th",Running,17.6,null,null,null,null


In [48]:
whoop.to_csv('../Data/whoop_data.csv',index=False)

In [199]:
def next_page(tod):
    yesterday=tod-datetime.timedelta(1)
    yesterdate=yesterday.strftime('%Y-%m-%d')
    browser.get('https://app.whoop.com/athlete/24590/strain/1d/{}'.format(yesterdate))
    tod=yesterday
    return tod

In [202]:
whoop=pd.DataFrame(columns=['strain','recovery','sleep_perf','sleep','rec_sleep','date', 
                            'activity_1','activity_1_score','activity_2','activity_2_score',
                           'activity_3','activity_3_score'])
tod=datetime.datetime.today()
num=0
old_date='today'
old_scores=['none']
old_strain=['null']
go_to_strain()
new_date=get_date(old_date)
while (new_date[0].strip().lower()!='wed, mar 27th' and num<10):
    new_date=get_date(old_date)
    print('made it')
    new_score=get_scores(old_scores)
    strains=get_activity_strain(old_strain,old_date)
    old_date=new_date
    old_scores=new_score
    old_strain=strains
    whoop.loc[len(whoop)]=new_score[:5]+new_date+strains
    tod=next_page(tod)
    num+=1

made it
made it
made it
made it
made it
made it


In [195]:
tod

In [150]:
new_date=get_date(old_date)

In [151]:
new_date

['Sun, Mar 31st']

In [152]:
new_score=get_scores(old_scores)
strains=get_activity_strain(old_strain,old_date)

In [155]:
old_date=new_date
old_scores=new_score
old_strain=strains
whoop.loc[len(whoop)]=new_score[:5]+new_date+strains

In [156]:
whoop

,strain,recovery,sleep_perf,sleep,rec_sleep,date,activity_1,activity_1_score,activity_2,activity_2_score,activity_3,activity_3_score
0,16.9,37%,75%,5:56,7:55,"Today, Apr 1st",Running,14.1,null,null,null,null
1,9.9,80%,100%,9:54,9:39,"Sun, Mar 31st",null,null,null,null,null,null


In [ ]:
findings[0].get_attribute('innerHTML')

In [213]:
whoop=pd.DataFrame(c`aolumns=['strain','recovery','sleep_perf','sleep','rec_sleep','date'])
num=0
date=['a']
while date[0].strip()!='Tue, May 29th' or num<300:
    browser.implicitly_wait(10)
    try:
        
        day = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "datepicker--label")))
    except TimeoutException:
        print("It broke :(")
    try:
        scores = WebDriverWait(browser, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "score")))
    except TimeoutException:
        print("It broke here :(")
    date=[day.text]
    score_list=[x.text for x in scores]
    old_scores=scores
    old_date=day.text
    whoop.loc[len(whoop)]=score_list[:5]+date
    back=browser.find_element_by_class_name('datepicker--prev')
    back.click()
    num+=1
    print(date[0])

Mon, Dec 31st


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.43.600229 (3fae4d0cda5334b4f533bede5a4787f7b832d052),platform=Mac OS X 10.14.1 x86_64)


In [214]:
whoop=pd.DataFrame(columns=['strain','recovery','sleep_perf','sleep','rec_sleep','date'])
num=0
date=['a']
while date[0].strip()!='Tue, May 29th' or num<300:
    browser.implicitly_wait(10)
    day=browser.find_element_by_class_name("datepicker--label")
    date=[day.text]
    scores=browser.find_elements_by_class_name('score')
    score_list=[x.text for x in scores]
    whoop.loc[len(whoop)]=score_list[:5]+date
    back=browser.find_element_by_class_name('datepicker--prev')
    back.click()
    num+=1
    print(date[0])

Sun, Dec 30th


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.43.600229 (3fae4d0cda5334b4f533bede5a4787f7b832d052),platform=Mac OS X 10.14.1 x86_64)


In [202]:
scores

[<selenium.webdriver.remote.webelement.WebElement (session="d07f868c969de8b3a0ae540d52d821a8", element="0.7655306361203951-2")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d07f868c969de8b3a0ae540d52d821a8", element="0.7655306361203951-41")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d07f868c969de8b3a0ae540d52d821a8", element="0.7655306361203951-42")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d07f868c969de8b3a0ae540d52d821a8", element="0.7655306361203951-43")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d07f868c969de8b3a0ae540d52d821a8", element="0.7655306361203951-6")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d07f868c969de8b3a0ae540d52d821a8", element="0.7655306361203951-53")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d07f868c969de8b3a0ae540d52d821a8", element="0.7655306361203951-56")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d07f868c969de8b3a0ae540d52d821a8

In [144]:
months=browser.find_element_by_xpath('/html/body/ui-view/div[4]/main/ui-view/div/ui-view/div/div[2]/div[1]/duration-picker/div/section[2]/button[1]')
months.click()

In [40]:
data=browser.page_source

In [147]:
cont=BS(data,'html.parser')

In [68]:
import re

In [82]:
re.search("\d{3}",'left: 115px; top: 264px;').group()

'115'

In [149]:
graphs=cont.find_all('graph-points', attrs={'graph-series':['recoverySeries','strainSeries']})
points=[]
for d in graphs[0].find_all('div'):
    px=re.search("(\d{3}|\d{2})",d.get('style')).group()
    points.append(px)
    print(d)

<div class="point recovery strain-strenuous recovery-medium" style="left: 0px; top: 180px;"></div>
<div class="point recovery strain-minimal recovery-low" style="left: 5px; top: 275px;"></div>
<div class="point recovery strain-all-out recovery-low" style="left: 9px; top: 257px;"></div>
<div class="point recovery strain-strenuous recovery-low" style="left: 14px; top: 250px;"></div>
<div class="point recovery strain-minimal recovery-low" style="left: 18px; top: 246px;"></div>
<div class="point recovery strain-moderate recovery-low" style="left: 23px; top: 239px;"></div>
<div class="point recovery strain-moderate recovery-low" style="left: 28px; top: 271px;"></div>
<div class="point recovery strain-moderate recovery-high" style="left: 32px; top: 113px;"></div>
<div class="point recovery strain-strenuous recovery-medium" style="left: 37px; top: 141px;"></div>
<div class="point recovery strain-moderate recovery-medium" style="left: 41px; top: 197px;"></div>
<div class="point recovery strain

In [146]:
hover=browser.find_element_by_tag_name('graph-tracker')#.find_element_by_tag_name('svg')
#browser.execute_script("arguments[0].set_attribute(style)")

# hover=browser.find_element_by_tag_name('graph-tracker')#.find_element_by_tag_name('svg')
# hover.get_attribute("innerHTML")



AttributeError: 'WebElement' object has no attribute 'load'

<svg class="hovered tracker hidden" style="width: 120px; transform: translateX(143px);">
  <line x1="42.15" x2="42.15" y1="0" y2="100%" stroke-width="1"></line>
<circle cx="42.15" r="5.3" cy="158" class="recovery strain-moderate recovery-medium"></circle><circle cx="42.15" r="5.3" cy="145" class="strain strain-moderate recovery-medium"></circle></svg>

<svg class="hovered tracker hidden" style="width: 120px; transform: translateX(143px);">
  <line x1="42.15" x2="42.15" y1="0" y2="100%" stroke-width="1"></line>
<circle cx="42.15" r="5.3" cy="158" class="recovery strain-moderate recovery-medium"></circle><circle cx="42.15" r="5.3" cy="145" class="strain strain-moderate recovery-medium"></circle></svg>

In [ ]:
<svg class="hovered tracker hidden" style="width: 120px; transform: translateX(143px);">
  <line x1="42.15" x2="42.15" y1="0" y2="100%" stroke-width="1"></line>
<circle cx="42.15" r="5.3" cy="158" class="recovery strain-moderate recovery-medium"></circle><circle cx="42.15" r="5.3" cy="145" class="strain strain-moderate recovery-medium"></circle></svg>

In [128]:
hover=browser.find_element_by_tag_name('graph-tracker').find_element_
#browser.execute_sc`ript(hover.)


#browser.execute_script("browser.find_element_by_xpath("
                       #("title").innerHTML = "New text!";"")

SyntaxError: unexpected EOF while parsing (<ipython-input-128-94bdea4a75cc>, line 4)

In [106]:
hover.get_attribute('innerHTML')

'\n  <line x1="42.15" x2="42.15" y1="0" y2="100%" stroke-width="1"></line>\n<circle cx="42.15" r="5.3" cy="180" class="recovery strain-strenuous recovery-medium"></circle><circle cx="42.15" r="5.3" cy="43" class="strain strain-strenuous recovery-medium"></circle>'

In [ ]:
url="https://www.strava.com/oauth/token"
rest="&grant_type:refresh_token&refresh_token:e2aca17c0295ff2d460f3ca1755c354fa15efd82"

In [19]:
r=requests.get(url+rest)

In [20]:
CLIENT_ID='15756'
CLIENT_SECRET='4fa0d8ee1b178973ce553031c075f25bd22e9abe'
tok='e2aca17c0295ff2d460f3ca1755c354fa15efd82'

In [73]:
def get_token(ref_token):
    client_auth = requests.auth.HTTPBasicAuth(CLIENT_ID, CLIENT_SECRET)
    post_data = {"grant_type": "refresh_token",
                "refresh_token": ref_token}
    response = requests.post("https://www.strava.com/oauth/token",
                            auth=client_auth,
                            data=post_data)
    token_json = response.json()
    return token_json

In [74]:
get_token(tok)

{'errors': [{'code': 'invalid',
   'field': 'client_id',
   'resource': 'Application'}],
 'message': 'Bad Request'}

In [27]:
# Initialize the dataframe
col_names = ['id','type']
activities = pd.DataFrame(columns=col_names)

access_token = "access_token=e2aca17c0295ff2d460f3ca1755c354fa15efd82" # replace with your access token here
url = "https://www.strava.com/api/v3/activities"

In [28]:
page = 1

while True:
    
    # get page of activities from Strava
    r = requests.get(url + '?' + access_token + '&per_page=50' + '&page=' + str(page))
    r = r.json()

    # if no results then exit loop
    if (not r):
        break
    
    # otherwise add new data to dataframe
    for x in range(len(r)):
        activities.loc[x + (page-1)*50,'id'] = r[x]['id']
        activities.loc[x + (page-1)*50,'type'] = r[x]['type']

    # increment page
    page += 1

KeyError: 0